In [1]:
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
import os
import json

In [2]:
path = '../input/ruthenia'
names = os.listdir(os.path.realpath(path))

In [ ]:
def html_parser(names, path):
    partial = SoupStrainer('div', id='main')
    finialy = {}
    for name in names:
        npath = os.path.realpath(path) + '\\' + name
        
        with open(npath, 'r', encoding='utf-8') as file_open:
            try:
                html_obj = BeautifulSoup(file_open, 'html.parser', parse_only=partial, from_encoding='utf-8')
#                 print(html_obj.prettify(), '\n')
            
                dict_of_stories = {}
                motive = html_obj.find('p', {'class': 'NormalLin'}).get_text()
                description = html_obj.find('p', {'class': 'NormalLis'}).get_text()
                stories = html_obj.find_all('p', {'class': 'NormalMai'})
                dict_of_stories['motive'] = str(motive)
                dict_of_stories['motive_description'] = str(description)

                country_dict = {}
                for num, story in enumerate(stories):
                    full_dict = {}
                    if story.b:
                        region = story.b
                        if isinstance(region.next_element.next_element, str):
                            region_next = region.next_element
                            region_text1 = story.b.get_text()
                            region_text2 = region.next_element.next_element
                        else:
                            region_next = region
                            region_text1 = story.b.get_text()
                            region_text2 = None 
                    else:
                        region = None
                    if region:                    
                        def next_u(data):
                            if data.find_next_sibling('u'):
                                return data.find_next_sibling('u'), data.find_next_sibling('u').next_element.next_element
                            else: return None, None
                        def next_i(data):
                            if data.find_next_sibling('i'):
                                return (data.find_next_sibling('i').find_previous_sibling('u'),
                                         data.find_next_sibling('i'),
                                         data.find_next_sibling('i').next_element.next_element)
                            else: return None, None, None

                        dict_of_countries = {}
                        counter_of_countries = 0
                        data_u = region
                        while data_u:
                            dict_of_people = {}
                            data_u, data_u_text = next_u(data_u)
                            if data_u:
                                dict_of_people['people'] = str(data_u.get_text())
                                dict_of_people['people_description'] = str(data_u_text)
                                dict_of_countries['country_content_' + str(counter_of_countries)] = dict_of_people                            
                            counter_of_deeper = 0
                            data_i = region
                            while data_i:
                                dict_of_substory = {}
                                data_u_i, data_i, data_i_text = next_i(data_i)
                                if data_u_i and data_u_i == data_u:
                                    dict_of_substory['substory'] = str(data_i.get_text())
                                    dict_of_substory['substory_description'] = str(data_i_text)
                                    dict_of_people['people_content_' + str(counter_of_deeper)] = dict_of_substory
                                counter_of_deeper += 1
                            counter_of_countries += 1

                    full_dict['country'] = str(region_text1)
                    full_dict['country_description'] = str(region_text2)
                    full_dict['country_content'] = dict_of_countries                
                    country_dict['story_' + str(num)] = full_dict

                dict_of_stories['motive_content'] = country_dict          

                finialy[name] = dict_of_stories
#                 print(finialy)
            except AttributeError:
                print('{0} cant\'be opened as partial'.format(name))
            except UnicodeDecodeError:
                print('{0} codec problem'.format(name))            

#         yield print('ok') #для тестов
    return finialy


In [6]:
def html_parser_v01(names, path):
    partial = SoupStrainer('div', id='main')
    finialy_list = []
    for name in names:
        npath = os.path.realpath(path) + '\\' + name        
        
        with open(npath, 'r', encoding='utf-8') as file_open:
            try:
                html_obj = BeautifulSoup(file_open, 'html.parser', parse_only=partial, from_encoding='utf-8')
            
                finaly_dict = {}
                
                finaly_dict['file'] = name
                print('file:\n{0}\n'.format(name))
                
                motive = html_obj.find('p', {'class': 'NormalLin'}).get_text()
                finaly_dict['motive'] = motive
                print('motive:\n{0}\n'.format(motive))
                
                description = html_obj.find('p', {'class': 'NormalLis'}).stripped_strings
                motive_description = []
                for desc in description:
                    if desc:
                        motive_description.append(desc)
                if motive_description:
                    finaly_dict['motive_description'] = motive_description                    
                else:
                    finaly_dict['motive_description'] = None
                print('motive_description:\n{0}\n'.format(motive_description))
                
                contents_list = []
                contents = html_obj.find_all('p', {'class': 'NormalMai'})
                for content in contents:
                    
                    regions = content.find_all('b')              
                    for num, val in enumerate(regions):
                        val['id'] = num
                    
                    regions_dict = {}
                    for reg in range(len(regions)):
                        region = content.find(id=reg)
                        region_descriprtion = content.find(id=reg).next_element.next_element
                        print('region, region_descriprtion:\n{0}\n{1}\n'.format(region.get_text(), region_descriprtion))
                        
                        region_contents = region.find_next_siblings('u')                        
                        for num, val in enumerate(region_contents):
                            val['id'] = num
                            
                        regions_cont_dict = {}
                        for pep in range(len(region_contents)):
                            people = content.find(id=pep)
                            people_descriprtion = content.find(id=pep).next_element.next_element
                            print('people, people_descriprtionn:\n{0}\n{1}\n'.format(people.get_text(), 
                                                                                     people_descriprtion))
                    
                    print('content:\n{0}\n'.format(content) )        

                    yield print('ok') #для тестов

            except AttributeError:
                print('{0} cant\'be opened as partial'.format(name))
            except UnicodeDecodeError:
                print('{0} codec problem'.format(name))            

#             yield print('ok') #для тестов
#     return finialy

In [7]:
parsed = html_parser_v01(names, path)

In [8]:
next(parsed)

file:
a10.html

motive:
A10. Солнце находит себе глаза. .52.61.

motive_description:
['Солнце получает свои сверкающие глаза (глаз) от животного.']

region, region_descriprtion:
Мезоамерика. 
Солнце и луна убивают чудовищного змея или орла и забирают себе его сверкающие глаза. 

people, people_descriprtionn:
Мезоамерика. 
Солнце и луна убивают чудовищного змея или орла и забирают себе его сверкающие глаза. 

people, people_descriprtionn:
трики
 [см. мотив A20; братья-близнецы живут в доме старухи, убивают ее мужа-Оленя; змея собирается сожрать мир, близнецы бросают ей в пасть раскаленные камни; посылают муху проверить, мертва ли змея; муха испражняется на ее левый глаз; младший брат берет себе более яркий правый глаз, старший – левый; братья идут, младший хочет пить; старший добывает воду как только младший соглашается поменяться глазами; Г. приходит к близнецам; они ее усыпляют и насилуют, нацепив камни на пенисы; убегают на небо, старший становится Солнцем, младший – Месяцем; он прог

In [9]:
next(parsed)

region, region_descriprtion:
Ср. Гондурас – Панама.
 

people, people_descriprtionn:
Ср. Гондурас – Панама.
 

content:
<p class="NormalMai">(<b id="0">Ср. Гондурас – Панама.</b> <u id="0">Брибри</u> [собираясь принести в жертву девочку Iriria, чтобы сделать из нее землю, Sibö послал муху позвать царя стервятников, прославленного певца и танцора; за это дал мухе зеленую радужную расцветку; Стервятник пел и плясал, затем попросил в награду какао; для сверхъестественных существ какао – это кровь людей; С. предложил ему сперва пить чичу, напоил допьяна, вынул его зоркие опасные глаза, подменил более слабыми, сделанными из семян дерева slá; разрешил в награду за танцы питаться глазами и плотью умерших]: Segundo Vargas 1997: 58).</p>

ok


In [10]:
next(parsed)

region, region_descriprtion:
Западная Амазония.
 Солнце выменивает у лиса зоркие и сверкающие глаза на свои слабые. 

people, people_descriprtionn:
Западная Амазония.
 Солнце выменивает у лиса зоркие и сверкающие глаза на свои слабые. 

people, people_descriprtionn:
агуаруна
: Akutz Nugkai et al. 1977(1) [Солнце и Лис меняются глазами во время купания], № 10, 11 [одалживает глаза Лиса, чтобы поймать рыбу]: 233-238.

content:
<p class="NormalMai"><b id="0">Западная Амазония.</b> Солнце выменивает у лиса зоркие и сверкающие глаза на свои слабые. <u id="0">Ачуар</u> [Солнце, Месяц и Лис купаются в реке, оставляют глаза на берегу; Солнце выходит первым, уносит глаза Лиса; теперь у него яркие глаза, а Лису пришлось взять обыкновенные, которые были у Солнца]: Mowitz 1978: 17; <u id="1">агуаруна</u>: Akutz Nugkai et al. 1977(1) [Солнце и Лис меняются глазами во время купания], № 10, 11 [одалживает глаза Лиса, чтобы поймать рыбу]: 233-238.</p>

ok


In [66]:
next(parsed)

<div id="main">
 <p class="NormalLin">
  A10. Солнце находит себе глаза. .52.61.
 </p>
 <p class="NormalLis">
  Солнце получает свои сверкающие глаза (глаз) от животного.
 </p>
 <p class="NormalMai">
  <b>
   Мезоамерика.
  </b>
  Солнце и луна убивают чудовищного змея или орла и забирают себе его сверкающие глаза.
  <u>
   Чатино
  </u>
  [змей]: Bartolomé 1979, № 2: 25; 1984: 12; Carrasco 1961: 46;
  <u>
   трики
  </u>
  [см. мотив A20; братья-близнецы живут в доме старухи, убивают ее мужа-Оленя; змея собирается сожрать мир, близнецы бросают ей в пасть раскаленные камни; посылают муху проверить, мертва ли змея; муха испражняется на ее левый глаз; младший брат берет себе более яркий правый глаз, старший – левый; братья идут, младший хочет пить; старший добывает воду как только младший соглашается поменяться глазами; Г. приходит к близнецам; они ее усыпляют и насилуют, нацепив камни на пенисы; убегают на небо, старший становится Солнцем, младший – Месяцем; он проглотил кролика, тот те

In [67]:
next(parsed)

region, region_descriprtion: Ср. Гондурас – Панама.,  

content: <p class="NormalMai">(<b id="0">Ср. Гондурас – Панама.</b> <u>Брибри</u> [собираясь принести в жертву девочку Iriria, чтобы сделать из нее землю, Sibö послал муху позвать царя стервятников, прославленного певца и танцора; за это дал мухе зеленую радужную расцветку; Стервятник пел и плясал, затем попросил в награду какао; для сверхъестественных существ какао – это кровь людей; С. предложил ему сперва пить чичу, напоил допьяна, вынул его зоркие опасные глаза, подменил более слабыми, сделанными из семян дерева slá; разрешил в награду за танцы питаться глазами и плотью умерших]: Segundo Vargas 1997: 58).</p>

ok


In [68]:
next(parsed)

region, region_descriprtion: Западная Амазония.,  Солнце выменивает у лиса зоркие и сверкающие глаза на свои слабые. 

content: <p class="NormalMai"><b id="0">Западная Амазония.</b> Солнце выменивает у лиса зоркие и сверкающие глаза на свои слабые. <u>Ачуар</u> [Солнце, Месяц и Лис купаются в реке, оставляют глаза на берегу; Солнце выходит первым, уносит глаза Лиса; теперь у него яркие глаза, а Лису пришлось взять обыкновенные, которые были у Солнца]: Mowitz 1978: 17; <u>агуаруна</u>: Akutz Nugkai et al. 1977(1) [Солнце и Лис меняются глазами во время купания], № 10, 11 [одалживает глаза Лиса, чтобы поймать рыбу]: 233-238.</p>

ok


In [ ]:
parsed = html_parser(names, path)

In [ ]:
parsed

In [ ]:
with open('../output/ruthenia_parsed.json', 'w') as f:
    json.dump(parsed, f)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
extracted_json = pd.read_json('../output/ruthenia_parsed.json', orient='index')

In [ ]:
extracted_json

In [ ]:
extracted_json = extracted_json.rename_axis('file').reset_index()

In [ ]:
extracted_json

In [ ]:
extracted_json['motive_content']

In [ ]:
normalised1 = pd.io.json.json_normalize(extracted_json['motive_content'],
                                             errors="ignore")

In [ ]:
normalised1

In [ ]:
normalised1.loc[115].dropna()

In [ ]:
type(normalised1.loc[115].dropna())

In [ ]:
normalised1.loc[115].dropna().loc['story_1.country_content.country_content_0.people_description']

In [ ]:
# тестирую разбор html на генераторной функции
next(parsed)